In [1]:
import pandas as pd

In [9]:
df_elec_consumption = pd.read_csv("cia_country_electricity_consupmtion.csv")
df_elec_consumption = df_elec_consumption.drop(columns=['Unnamed: 0','year','Unnamed: 4',])
df_elec_consumption.head()

,country,electricity_consumption_kWh
0,China,"5,564,000,000,000"
1,United States,"3,902,000,000,000"
2,India,"1,137,000,000,000"
3,Japan,"943,700,000,000"
4,Russia,"909,600,000,000"


In [10]:
df_emissions = pd.read_csv("cia_country_emissions.csv")
df_emissions = df_emissions.drop(columns=['Unnamed: 0','year','Unnamed: 4',])
df_emissions.head()

,country,carbon_emissions_from_energy_consumption_MT
0,China,"11,670,000,000.00"
1,United States,"5,242,000,000.00"
2,India,"2,383,000,000.00"
3,Russia,"1,847,000,000.00"
4,Japan,"1,268,000,000.00"


In [11]:
df_fossil_elec = pd.read_csv("cia_country_fossil_electricity.csv")
df_fossil_elec = df_fossil_elec.drop(columns=['Unnamed: 0','year','Unnamed: 4',])
df_fossil_elec.head()

,country,fossil_fuel_percentage_of_electricity_production
0,"Bahamas, The",100
1,Bahrain,100
2,Bermuda,100
3,Botswana,100
4,Brunei,100


In [12]:
df_hydro_elec = pd.read_csv("cia_country_hydro_electricity.csv")
df_hydro_elec = df_hydro_elec.drop(columns=['Unnamed: 0','year','Unnamed: 4',])
df_hydro_elec.head()

,country,hydro_percentage_of_electricity_production
0,Lesotho,100
1,Bhutan,99
2,Paraguay,99
3,"Congo, Democratic Republic of the",98
4,Albania,95


In [16]:
df_internet_users = pd.read_csv("cia_country_internet_users.csv")
df_internet_users = df_internet_users.drop(columns=['Unnamed: 0','Unnamed: 3',])
df_internet_users.head()

,country,number_of_internet_users
0,China,"730,723,960"
1,India,"374,328,160"
2,United States,"246,809,221"
3,Brazil,"122,841,218"
4,Japan,"116,565,962"


In [17]:
df_gas_consumption = pd.read_csv("cia_country_natural_gas_consumption.csv")
df_gas_consumption = df_gas_consumption.drop(columns=['Unnamed: 0','year','Unnamed: 4',])
df_gas_consumption.head()

,country,natural_gas_consumption_cubic_meters
0,United States,"767,600,000,000"
1,Russia,"467,500,000,000"
2,China,"238,600,000,000"
3,Iran,"206,900,000,000"
4,Japan,"127,200,000,000"


In [18]:
df_nuclear_elec = pd.read_csv("cia_country_nuclear_electricity.csv")
df_nuclear_elec = df_nuclear_elec.drop(columns=['Unnamed: 0','year','Unnamed: 4',])
df_nuclear_elec.head()

,country,nuclear_percentage_of_electricity_production
0,France,50
1,Belgium,28
2,Slovakia,27
3,Ukraine,23
4,Hungary,22


In [19]:
df_other_renewables = pd.read_csv("cia_country_other_renewable_electricity.csv")
df_other_renewables = df_other_renewables.drop(columns=['Unnamed: 0','year','Unnamed: 4',])
df_other_renewables.head()

,country,other_renewable_percentage_of_electricity_production
0,Timor-Leste,100
1,Luxembourg,67
2,Denmark,54
3,Germany,52
4,Uruguay,42


In [20]:
df_ref_petrol_consumption = pd.read_csv("cia_country_refined_petroleum_consumption.csv")
df_ref_petrol_consumption = df_ref_petrol_consumption.drop(columns=['Unnamed: 0','year','Unnamed: 4',])
df_ref_petrol_consumption.head()

,country,refined_petroleum_consumption_barrels_per_day
0,United States,"19,960,000"
1,China,"12,470,000"
2,India,"4,521,000"
3,Japan,"3,894,000"
4,Russia,"3,650,000"
